In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras import optimizers
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from sklearn.metrics import classification_report,confusion_matrix

import warnings
warnings.filterwarnings('ignore')

from timeit import default_timer as timer
from datetime import timedelta

In [ ]:
src_dir = os.path.join('C:\\Users\\username\\Projects\\project', 'hands')

In [ ]:
test_path = os.path.join(src_dir, 'test')
train_path = os.path.join(src_dir, 'train')

In [ ]:
dimension1 = []
dimension2 = []
for image in os.listdir(os.path.join(test_path, 'real')):
    
    img = imread(os.path.join(test_path, 'real', image))
    d1,d2,colors = img.shape
    dimension1.append(d1)
    dimension2.append(d2)

In [ ]:
img_shape = (140,130,3)

In [ ]:
image_generator = ImageDataGenerator(rotation_range=20, 
                               width_shift_range=0.10, 
                               height_shift_range=0.10, 
                               rescale=1/255, 
                               shear_range=0.1, 
                               zoom_range=0.2, 
                               horizontal_flip=True, 
                               vertical_flip=True, 
                               fill_mode='constant' 
                              )

In [ ]:
vgg = VGG16(input_shape = img_shape, weights = 'imagenet', include_top = False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
flat_layer = Flatten()(vgg.output)
pred = Dense(1, activation = 'sigmoid')(flat_layer)
model = Model(inputs = vgg.input, outputs = pred)
model.summary()

In [ ]:
opt = optimizers.Adam(learning_rate=1e-04)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
batch_size = 256

In [ ]:
train_image_gen = image_generator.flow_from_directory(train_path,
                                               target_size=img_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_gen = image_generator.flow_from_directory(test_path,
                                               target_size=img_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

In [ ]:
train_image_gen.class_indices

In [ ]:
start = timer()

In [ ]:
results = model.fit_generator(train_image_gen,epochs=100,
                              validation_data=test_image_gen,
                             callbacks=[early_stop])

In [ ]:
end = timer()
print(timedelta(seconds=end-start))

In [ ]:
model.save('hand_fake_detector_transfer_lerning.h5')

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
model =  load_model('hand_fake_detector_transfer_lerning.h5')

In [ ]:
test_image_gen.class_indices

In [ ]:
pred_probabilities = model.predict_generator(test_image_gen)

In [ ]:
predictions = pred_probabilities > 0.5

In [ ]:
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)